# Welcome to the OnSSET Notebook

This Jupyter based interface is built on the [OnSSET](http://www.onsset.org/) tool developed to provide an easy and quick way to generate electrification investment scenarios.

#### Start by importing the code 

In [ ]:
from onsset import *
from IPython.display import display, Markdown, HTML
%matplotlib inline
%run funcs.ipynb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib.pylab as plt
import seaborn as sns

# 1. GIS data selection

First, run the cell below to browse to the directory your input CSV file is located at and select the input file. 

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)
messagebox.showinfo('OnSSET', 'Open the input file with extracted GIS data')
input_file = filedialog.askopenfilename()

onsseter = SettlementProcessor(input_file)
onsseter.conditioning()

In [ ]:
messagebox.showinfo('OnSSET', 'Open the file with hourly PV data')
pv_path = filedialog.askopenfilename()
messagebox.showinfo('OnSSET', 'Open the file with hourly Wind data')
wind_path = filedialog.askopenfilename()

In [ ]:
messagebox.showinfo('OnSSET', 'Open the existing MV lines')
existing_mv = filedialog.askopenfilename()
x_mv_exist, y_mv_exist = onsseter.start_extension_points(existing_mv)
x_coordinates = x_mv_exist
y_coordinates = y_mv_exist

# 2. Modelling period and target electrification rate

Next, define the modelling period and the electrification rate to be achieved by the end of the analysis. Further down you will also define an intermediate year and target.

In [ ]:
start_year = 2023
intermediate_year = 2027
end_year = 2030

intermediate_electrification_rate_target = 0.80 # E.g. 1 for 100% electrification rate or 0.80 for 80% electrification rate
end_electrification_rate_target = 1 # E.g. 1 for 100% electrification rate or 0.80 for 80% electrification rate 

yearsofanalysis = [intermediate_year, end_year]
eleclimits = {intermediate_year: intermediate_electrification_rate_target, end_year: end_electrification_rate_target}
time_steps = {intermediate_year: intermediate_year-start_year, end_year: end_year-intermediate_year}

# 3. Enter country specific data

In addition to the options above the user can customize a large number of variables describing the social - economic - technological environment in the selected country. 

**Note!** Most input values shall represent estimates for the variable valid throughout the modelling period, i.e. **NOT** current values.

### a. Demographics and Social components

In [ ]:
pop_start_year = 32419747       ### Write the population in the base year (e.g. 2020) 
end_year_pop = 38689102         ### Write the expected population in the end year (e.g. 2030) 

urban_ratio_start_year = 0.32 ### Write the urban population population ratio in the base year (e.g. 2020)
urban_ratio_end_year = 0.36     ### Write the urban population population ratio in the end year (e.g. 2030)

num_people_per_hh_urban = 5     ### Write the number of people per household expected in the end year (e.g. 2030)
num_people_per_hh_rural = 6   ### Write the number of people per household expected in the end year (e.g. 2030)

grid_elec_ratio_start_year = 0.48   ### Write the grid electrification rate in the base year (e.g. 2020)
grid_urban_elec_ratio = 0.91      ### Write urban grid electrification rate in the base year (e.g. 2020)
grid_rural_elec_ratio = 0.07         ### Write rural grid electrification rate in the base year (e.g. 2020)

### b. Technology specifications & costs

The cell below contains all the information that is used to calculate the levelized costs for all the technologies, including grid. These default values should be updated to reflect the most accurate values in the country. There are currently 6 potential technologies to include in the model:
 * Grid
 * 
PV Hybrid Mini-grid
 * Wind Mini-grid
 * Hydro Mini-grid
 * PV Stand-alone systems
 * Diesel stand-alone systems

#### Centralized grid parameters 

In [ ]:
grid_generation_cost = 0.10                ### This is the grid cost electricity USD/kWh as expected in the end year of the analysis
grid_power_plants_capital_cost = 2000      ### The cost in USD/kW is for capacity upgrades of the grid
grid_losses = 0.15                         ### The fraction of electricity lost in transmission and distribution (percentage)  

annual_new_grid_connections_limit = 9999999 # This is the maximum amount of new households that can be connected to the grid in one year
annual_grid_cap_gen_limit = 9999999        # This is the maximum generation capacity (MW) that can be added to the grid in one year

#### Off-grid technology parameters

In [ ]:
min_mg_size = 250             # Minimum number of connections (people) for mini-grids to be considered
mg_min_grid_dist = 10          # Minimum distance from existing MV lines for mini-grids to be considered as an option

# Decide whether mini-grids should be allowed to be interconnected to the grid in a later time-step.
mg_interconnection = 0         # 0 = NO, 1 = YES

In [ ]:
diesel_price = 1.43                    ### This is the diesel price in USD/liter as expected in the end year of the analysis

mg_diesel_cost = {'diesel_price': diesel_price,
                  'efficiency': 0.33,
                  'diesel_truck_consumption': 33.7,
                  'diesel_truck_volume': 15000}

In [ ]:
mg_hydro_capital_cost = {float("inf"): 3000}      ### Mini-grid Hydro capital cost (USD/kW) as expected in the years of the analysis

In [ ]:
# PV and Wind hybrid mini-grid costs
pv_cost = 2000                      # PV panel costs including BoS (PV inverter, charge controller) (USD/kW)
battery_cost = 148                 # battery capital cost, USD/kWh of storage capacity                    
inverter_cost  = 598             # Battery inverter, USD/kW
diesel_gen_cost = 1544              # diesel generator capital cost, USD/kW rated power
wind_cost = 2800                   # Wind turbine capital cost, USD/kW peak power

max_diesel = 0.5                   # Maximum share of generation that can come from diesel generators (0-1). Set to 0 for fully renewable mini-grids

In [ ]:
sa_pv_capital_cost_1 = 9620          ### Stand-alone PV capital cost (USD/kW) for household systems under 20 W
sa_pv_capital_cost_2 = 8780          ### Stand-alone PV capital cost (USD/kW) for household systems between 21-50 W
sa_pv_capital_cost_3 = 6380           ### Stand-alone PV capital cost (USD/kW) for household systems between 51-100 W
sa_pv_capital_cost_4 = 4470           ### Stand-alone PV capital cost (USD/kW) for household systems between 101-1000 W
sa_pv_capital_cost_5 = 4500           ### Stand-alone PV capital cost (USD/kW) for household systems over 1 kW

The cells below contain additional technology specifications

In [ ]:
grid_discount_rate = 0.08 # E.g. 0.08 means a discount rate of 8%
mini_grid_discount_rate = 0.08
standalone_discount_rate = 0.08

# Transmission and distribution costs
hv_line_capacity=132 # kV
hv_line_cost=112000 # USD/km

mv_line_capacity=33 # kV
mv_line_cost = 26000 # USD/kW
mv_line_max_length=150 # km
mv_line_amperage_limit = 8  # Ampere (A)

lv_line_capacity=0.400 #kV
lv_line_max_length=0.5 # km
lv_line_cost=18900 # USD/km

service_transformer_type=160  # kVA (MV/LV)
service_transformer_cost=29200  # $/unit
max_nodes_per_serv_trans=100  # maximum number of nodes served by each service (MV/LV) transformer

hv_mv_transformer_type = 40000 #kVA
hv_mv_transformer_cost = 1900000 # USD/unit

# Centralized grid costs
grid_calc = Technology(om_of_td_lines=0.02,
                        distribution_losses=grid_losses,
                        connection_cost_per_hh=100,
                        base_to_peak_load_ratio=0.8,
                        capacity_factor=1,
                        tech_life=30,
                        grid_penalty_ratio=1,
                        grid_capacity_investment=grid_power_plants_capital_cost,
                        grid_price=grid_generation_cost,
                        discount_rate=grid_discount_rate)

mg_pv_hybrid_calc = Technology(om_of_td_lines=0.02, 
                               distribution_losses=0.05, 
                               connection_cost_per_hh=100, 
                               capacity_factor=0.5, 
                               base_to_peak_load_ratio=0.85,
                               tech_life=20, 
                               mini_grid=True,  
                               hybrid=True, 
                               discount_rate=mini_grid_discount_rate)

mg_wind_hybrid_calc = Technology(om_of_td_lines=0.02,
                          distribution_losses=0.05,
                          connection_cost_per_hh=100,
                          capacity_factor=0.5,
                          base_to_peak_load_ratio=0.85,
                          tech_life=20,
                          mini_grid=True,
                          hybrid=True,
                          discount_rate=mini_grid_discount_rate)

# Mini-grid hydro costs
mg_hydro_calc = Technology(om_of_td_lines=0.02,
                            distribution_losses=0.05,
                            connection_cost_per_hh=100,
                            base_to_peak_load_ratio=0.85,
                            capacity_factor=0.5,
                            tech_life=30,
                            capital_cost=mg_hydro_capital_cost,
                            om_costs=0.02,
                            discount_rate=mini_grid_discount_rate)
                            

# Stand-alone PV costs
sa_pv_calc = Technology(base_to_peak_load_ratio=0.9,
                        tech_life=5,
                        om_costs=0.05,
                        capital_cost={0.020: sa_pv_capital_cost_1, 
                                      0.050: sa_pv_capital_cost_2, 
                                      0.100: sa_pv_capital_cost_3, 
                                      1: sa_pv_capital_cost_4, 
                                      float("inf"): sa_pv_capital_cost_5},
                        standalone=True,
                        discount_rate=standalone_discount_rate)

mg_pv_hybrid_params = {
                'min_mg_size_ppl': min_mg_size,  # minimum number of people in settlement for mini-grids to be considered as an option
                'diesel_cost': diesel_gen_cost,  # diesel generator capital cost, USD/kW rated power
                'discount_rate': mini_grid_discount_rate,
                'n_chg': 0.93,  # charge efficiency of battery
                'n_dis': 1,  # discharge efficiency of battery
                'battery_cost': battery_cost,  # battery capital cost, USD/kWh of storage capacity
                'pv_cost': pv_cost,  # PV panel capital cost, USD/kW peak power
                'charge_controller': 0,  # PV charge controller cost, USD/kW peak power, set to 0 if already included in pv_cost
                'pv_inverter': 0,  # PV inverter cost, USD/kW peak power, set to 0 if already included in pv_cost
                'pv_life': 25,  # PV panel expected lifetime, years
                'diesel_life': 10,  # diesel generator expected lifetime, years
                'pv_om': 0.015,  # annual OM cost of PV panels
                'diesel_om': 0.1,  # annual OM cost of diesel generator
                'battery_inverter_cost': inverter_cost,
                'battery_inverter_life': 20,
                'dod_max': 0.8,  # maximum depth of discharge of battery
                'inv_eff': 0.93,  # inverter_efficiency
                'lpsp_max': 0.02,  # maximum loss of load allowed over the year, in share of kWh
                'diesel_limit': max_diesel,  # Max annual share of mini-grid generation from diesel gen-set
                'full_life_cycles': 2500  # Equivalent full life-cycles of battery until replacement
            }

mg_wind_hybrid_params = {
                'min_mg_size_ppl': min_mg_size,  # minimum number of people in settlement for mini-grids to be considered as an option
                'diesel_cost': diesel_gen_cost,  # diesel generator capital cost, USD/kW rated power
                'discount_rate': mini_grid_discount_rate,
                'n_chg': 0.93,  # charge efficiency of battery
                'n_dis': 1,  # discharge efficiency of battery
                'battery_cost': battery_cost,  # battery capital cost, USD/kWh of storage capacity
                'wind_cost': wind_cost,  # Wind turbine capital cost, USD/kW peak power
                'charge_controller': 0,  # PV charge controller cost, USD/kW peak power, set to 0 if already included in pv_cost
                'wind_life': 25,  # Wind turbine expected lifetime, years
                'diesel_life': 10,  # diesel generator expected lifetime, years
                'wind_om': 0.015,  # annual OM cost of wind turbine
                'diesel_om': 0.1,  # annual OM cost of diesel generator
                'battery_inverter_cost': inverter_cost,
                'battery_inverter_life': 20,
                'dod_max': 0.8,  # maximum depth of discharge of battery
                'inv_eff': 0.93,  # inverter_efficiency
                'lpsp_max': 0.02,  # maximum loss of load allowed over the year, in share of kWh
                'diesel_limit': max_diesel,  # Max annual share of mini-grid generation from diesel gen-set
                'full_life_cycles': 2500  # Equivalent full life-cycles of battery until replacement
            }

### c. Electricity demand target

For the second lever, enter the target tier (level of electricity access) for urban and rural households respectively. This can take a value between "1" (lowest level of electricity access) and "5" (highest level of electricity access) as in ESMAPs Multi-Tier Framework for Measuring Electricity Access (found <a href="https://www.esmap.org/node/55526" target="_blank">here</a>). Alternatively, enter "6" to use a distribution of the tiers across the country based on poverty levels and GDP according to the methodology found <a href="https://www.mdpi.com/1996-1073/12/7/1395" target="_blank">here</a>.   

*On the GEP Explorer, the following three electricity demand targets are used:*

*Top down demand target - Low: In this case, all urban clusters are tergeted to reach the current average consumption level of electrified households, and rural settlements are assigned Tier 1.* 

*Top down demand target - High: In this case, all urban clusters are tergeted to reach one Tier higher than the current average consumption level of electrified households, and rural settlements are assigned Tier 3.*

*Bottom up demand target (Poverty - GDP): In this case each settlement is assigned a demand target based on poverty and GDP levels as described in the methodology above. Choose "6" for both the urban_target_tier and rural_target_tier to use this option* 

In [ ]:
# Define the annual household electricity targets to choose from
tier_1 = 38.7  # 38.7 refers to kWh/household/year. 
tier_2 = 219
tier_3 = 803
tier_4 = 2117
tier_5 = 3000

In [ ]:
urban_target_tier = 4
rural_target_tier = 2

In [ ]:
productive_demand = 1                     # 1 if productive demand is defined and should be included, else 0

### d. Rollout plan
This lever reflects the electrification approach to be examined. On the GEP Explorer, there are currently two options in use:

**Nationwide Least Cost approach:** This options aims to achieve the electrification rate targets set for the intermediate and end year. For the years where the target is set below 100%, the algorithm prioritizes grid densification first (ramp up in already electrified clusters) then selection based on lowest invetsment cost per capita to choose which clusters to be electrified.

**Forced grid approach:** Forced grid under a defined buffer zone (auto_intensification set equal to X km) & least cost approach outside of the buffer zone.

In [ ]:
auto_intensification = 30        # Buffer distance (km) for automatic intensification

max_grid_intensification_cost = 2000  # Maximum cost per household (USD/household) for forced grid intensification 

min_intensification_load = 0  # Minimum peak load (kW) for forced grid intensification

# 4. Calibration of start year values and general information

The following steps calibrate the start year conditions in the country in terms of population and current electrification rate, and also adds some additional useful information to be used in the further calculations. If you have retrieved the input file from Energydata.info some of these steps are already completed, and you may skip those cells below where there is a note. 

In [ ]:
onsseter.df.loc[onsseter.df[SET_URBAN] == 0, SET_NUM_PEOPLE_PER_HH] = num_people_per_hh_rural
onsseter.df.loc[onsseter.df[SET_URBAN] == 1, SET_NUM_PEOPLE_PER_HH] = num_people_per_hh_rural
onsseter.df.loc[onsseter.df[SET_URBAN] == 2, SET_NUM_PEOPLE_PER_HH] = num_people_per_hh_urban

pop_modelled, urban_modelled = onsseter.calibrate_current_pop_and_urban(pop_start_year, urban_ratio_start_year)
onsseter.project_pop_and_urban(end_year_pop, urban_ratio_end_year, start_year, yearsofanalysis)
onsseter.df[SET_WINDCF] = onsseter.calc_wind_cfs(onsseter.df[SET_WINDVEL])

display(Markdown('#### The csv file has been imported correctly. Here is a preview:'))
display(onsseter.df[['Country','Pop', 'GridCellArea', 'NightLights', 'TravelHours', 'GHI', 'WindVel', 'CurrentMVLineDist']].sample(7))

#### Calibration of currently grid-electrified settlements

The model calibrates which settlements are likely to be electrified in the start year, to match the national statistical values defined above. A settlement is considered to be electrified if it meets all of the following conditions:
- Has more night-time lights than the defined threshold (this is set to 0 by default. Set to -1 to disregard NTL completely)
- Is closer to the existing grid network than the distance limit
- Has more population than the threshold

First, define the threshold limits. Then run the calibration and check if the results seem okay. Else, redefine these thresholds and run again.

In [ ]:
min_night_lights = 0.05    ### 0 Indicates no night light, while any number above refers to the night-lights intensity
min_pop = 250      ### Settlement population above which we can assume that it could be electrified

max_service_transformer_distance = 2    ### Distance  in km from the existing grid network below which we can assume a settlement could be electrified
max_mv_line_distance = 3
max_hv_line_distance = 5

elec_modelled, urban_internal_elec_ratio, rural_internal_elec_ratio, *_ = onsseter.calibrate_grid_elec_current(grid_elec_ratio_start_year, 
                                                                                                       grid_urban_elec_ratio, 
                                                                                                       grid_rural_elec_ratio, 
                                                                                                       start_year,
                                                                                                       min_night_lights=min_night_lights, 
                                                                                                       min_pop=min_pop, 
                                                                                                       max_transformer_dist=max_service_transformer_distance, 
                                                                                                       max_mv_dist=max_mv_line_distance, 
                                                                                                       max_hv_dist=max_hv_line_distance,
                                                                                                       buffer=False) 

print('Grid-electrified settlements: ', onsseter.df.ElecStart.sum())

In [ ]:
mg_dist = 1   # Distance from existing mini-grids to consider settlements connected to the mini-grid
mg_ntl = 0   # Night-time light threshold to consider a settlement mini-grid electrified, in combination with mg_dist. 0 means NTL is not required, any higher value means a higher cut-off for settlements with NTL within dist is electrified
mg_pop = 100  # Settlement population above which we can assume that it could be electrified by mini-grid

onsseter.df.loc[onsseter.df['FinalElecCode' + '{}'.format(start_year)] == 5, 'ElecStart'] = 0
onsseter.df.loc[onsseter.df['FinalElecCode' + '{}'.format(start_year)] == 5, 'FinalElecCode' + '{}'.format(start_year)] = 99
onsseter.df.loc[onsseter.df['FinalElecCode' + '{}'.format(start_year)] == 5, 'ElecPopCalib'] = 0
onsseter.df.loc[onsseter.df['FinalElecCode' + '{}'.format(start_year)] == 5, 'ElecPop' + '{}'.format(start_year)] = 0

onsseter.df.loc[(onsseter.df['FinalElecCode' + '{}'.format(start_year)] != 1) & (onsseter.df['NightLights'] >= mg_ntl) & 
                (onsseter.df['MGDist'] < mg_dist) & (onsseter.df[SET_POP + "{}".format(start_year)] > mg_pop), 'ElecStart'] = 1
onsseter.df.loc[(onsseter.df['FinalElecCode' + '{}'.format(start_year)] != 1) & (onsseter.df['NightLights'] >= mg_ntl) & 
                (onsseter.df['MGDist'] < mg_dist) & (onsseter.df[SET_POP + "{}".format(start_year)] > mg_pop), 'FinalElecCode' + '{}'.format(start_year)] = 5
onsseter.df.loc[(onsseter.df['FinalElecCode' + '{}'.format(start_year)] != 1) & (onsseter.df['NightLights'] >= mg_ntl) & 
                (onsseter.df['MGDist'] < mg_dist) & (onsseter.df[SET_POP + "{}".format(start_year)] > mg_pop), 'ElecPopCalib'] = onsseter.df['PopStartYear']
onsseter.df['ElecPop' + '{}'.format(start_year)] = onsseter.df['ElecPopCalib']

mg_pop = onsseter.df.loc[onsseter.df['FinalElecCode' + '{}'.format(start_year)] == 5, 'PopStartYear'].sum() / onsseter.df['PopStartYear'].sum()

print('The national modelled mini-grid electrification rate is {}'.format(round(mg_pop, 3)))
print('The modelled national electrification rate is {}'.format(round(mg_pop + elec_modelled, 3)))

#### Visualization

The figure below show the results of the calibration. Settlements in **blue** are considered to be (at least partly) grid-electrified already in the start year of the analysis, settlements in **purple** are considered to be connected to mini-grids at the start of the analysis, while settlements in **grey** are yet to be electrified. Re-running the calibration step with different intial values may change the map below.

In [ ]:
from matplotlib import pyplot as plt
colors = ['#D3D3D3','#808080', 'red']
plt.figure(figsize=(12,12))
plt.plot(onsseter.df.loc[onsseter.df['FinalElecCode' + '{}'.format(start_year)]==99, SET_X_DEG], onsseter.df.loc[onsseter.df['FinalElecCode' + '{}'.format(start_year)]==99, SET_Y_DEG], color='#D3D3D3', marker=',', linestyle='none')
plt.plot(onsseter.df.loc[onsseter.df['FinalElecCode' + '{}'.format(start_year)]==1, SET_X_DEG], onsseter.df.loc[onsseter.df['FinalElecCode' + '{}'.format(start_year)]==1, SET_Y_DEG], color='#00008B', marker=',', linestyle='none')
plt.plot(onsseter.df.loc[onsseter.df['FinalElecCode' + '{}'.format(start_year)]==5, SET_X_DEG], onsseter.df.loc[onsseter.df['FinalElecCode' + '{}'.format(start_year)]==5, SET_Y_DEG], color='red', marker=',', linestyle='none')
if onsseter.df[SET_X_DEG].max() - onsseter.df[SET_X_DEG].min() > onsseter.df[SET_Y_DEG].max() - onsseter.df[SET_Y_DEG].min():
    plt.xlim(onsseter.df[SET_X_DEG].min() - 1, onsseter.df[SET_X_DEG].max() + 1)
    plt.ylim((onsseter.df[SET_Y_DEG].min()+onsseter.df[SET_Y_DEG].max())/2 - 0.5*abs(onsseter.df[SET_X_DEG].max() - onsseter.df[SET_X_DEG].min()) - 1, (onsseter.df[SET_Y_DEG].min()+onsseter.df[SET_Y_DEG].max())/2 + 0.5*abs(onsseter.df[SET_X_DEG].max() - onsseter.df[SET_X_DEG].min()) + 1)
else:
    plt.xlim((onsseter.df[SET_X_DEG].min()+onsseter.df[SET_X_DEG].max())/2 - 0.5*abs(onsseter.df[SET_Y_DEG].max() - onsseter.df[SET_Y_DEG].min()) - 1, (onsseter.df[SET_X_DEG].min()+onsseter.df[SET_X_DEG].max())/2 + 0.5*abs(onsseter.df[SET_Y_DEG].max() - onsseter.df[SET_Y_DEG].min()) + 1)
    plt.ylim(onsseter.df[SET_Y_DEG].min() -1, onsseter.df[SET_Y_DEG].max() +1)
plt.figure(figsize=(30,30))
logging.getLogger('matplotlib.font_manager').disabled = True
plt.show()

# 5. Start a scenario run, which calculate and compare technology costs for every settlement in the country

Based on the previous calculation this piece of code identifies the LCoE that every off-grid technology can provide, for each single populated settlement of the selected country. The cell then takes all the currently grid-connected points in the country, and looks at the points within a certain distance from them, to see if it is more economical to connect them to the grid, or to use one of the off-grid technologies calculated above. Once more points are connected to the grid, the process is repeated, so that new points close to those points might also be connected. This is repeated until there are no new points to connect to the grid.

In [ ]:
x_coords = x_coordinates * 1
y_coords = y_coordinates * 1

import time
print('Starting', time.ctime())

techs = ["Grid", "SA_PV", "MG_Diesel", "MG_PVHybrid", "MG_Wind", "MG_Hydro"]
tech_codes = [1, 3, 4, 5, 6, 7]

onsseter.prepare_wtf_tier_columns(tier_1, tier_2, tier_3, tier_4, tier_5)

onsseter.current_mv_line_dist()

onsseter.add_xy_3395()

try:
    onsseter.df.reset_index(inplace=True)
except ValueError:
    pass

Technology.set_default_values(base_year=start_year, start_year=start_year, end_year=end_year,
                             hv_line_type=hv_line_capacity, hv_line_cost=hv_line_cost, mv_line_type=mv_line_capacity,
                             mv_line_amperage_limit=mv_line_amperage_limit, mv_line_cost=mv_line_cost, mv_line_max_length=mv_line_max_length,
                             lv_line_type=lv_line_capacity, lv_line_cost=lv_line_cost, lv_line_max_length=lv_line_max_length, 
                             service_transf_type=service_transformer_type, service_transf_cost = service_transformer_cost,
                             max_nodes_per_serv_trans=max_nodes_per_serv_trans, hv_mv_sub_station_cost=hv_mv_transformer_cost,
                             hv_mv_substation_type=hv_mv_transformer_type)

new_lines_geojson = {}

for year in yearsofanalysis:
        
    eleclimit = eleclimits[year]
    time_step = time_steps[year]
    
    grid_connect_limit = time_step * annual_new_grid_connections_limit
    grid_cap_gen_limit = time_step * annual_grid_cap_gen_limit * 1000
        
    onsseter.calculate_demand(year, num_people_per_hh_urban, num_people_per_hh_rural, time_step,
                                            urban_target_tier, rural_target_tier, moz=False)

    try:
        del onsseter.df[SET_MG_DIESEL_FUEL + "{}".format(year)]
        del onsseter.df[SET_SA_DIESEL_FUEL + "{}".format(year)]
    except:
        pass
    onsseter.diesel_cost_columns(mg_diesel_cost, mg_diesel_cost, year)
    mean = onsseter.df[SET_MG_DIESEL_FUEL + "{}".format(year)].mean()
    onsseter.df[SET_MG_DIESEL_FUEL + "{}".format(year)].fillna(mean, inplace=True)

    print('Optimize Wind Hybrid Systems', time.ctime())
    wind_hybrid_lcoe, wind_hybrid_capacity, wind_hybrid_investment, wind_check = \
                    onsseter.wind_hybrids_lcoe_lookuptable(year, time_step, end_year, mg_wind_hybrid_params, wind_path=wind_path)
    wind_hybrid_investment.fillna(0, inplace=True)
    wind_hybrid_capacity.fillna(0, inplace=True)

    mg_wind_hybrid_calc.hybrid_fuel = wind_hybrid_lcoe
    mg_wind_hybrid_calc.hybrid_investment=wind_hybrid_capacity
    mg_wind_hybrid_calc.hybrid_capacity=wind_hybrid_investment
        
    print('Optimize MG Hybrid Systems', time.ctime())
    hybrid_lcoe, hybrid_capacity, hybrid_investment, check = \
                    onsseter.pv_hybrids_lcoe_lookuptable(year, time_step, end_year, mg_pv_hybrid_params, pv_path=pv_path)
    hybrid_investment.fillna(0, inplace=True)
    hybrid_capacity.fillna(0, inplace=True)

    mg_pv_hybrid_calc.hybrid_fuel = hybrid_lcoe
    mg_pv_hybrid_calc.hybrid_investment=hybrid_investment
    mg_pv_hybrid_calc.hybrid_capacity=hybrid_capacity
    
    print('Calculate Off-Grid LCOEs', time.ctime())
    sa_diesel_investment, sa_diesel_capacity, sa_pv_investment, sa_pv_capacity, mg_diesel_investment, \
            mg_diesel_capacity, mg_pv_hybrid_investment, mg_pv_hybrid_capacity, mg_wind_investment, mg_wind_capacity, \
            mg_hydro_investment, mg_hydro_capacity = onsseter.calculate_off_grid_lcoes(mg_hydro_calc, mg_wind_hybrid_calc, sa_pv_calc, mg_pv_hybrid_calc,
                                                                                       year, end_year, time_step, techs, tech_codes,
                                                                                       min_mg_size, mg_min_grid_dist)
    
    print('Calculate Grid LCOEs', time.ctime())
    grid_investment, grid_capacity, grid_cap_gen_limit, grid_connect_limit = \
                onsseter.pre_electrification(grid_generation_cost, year, time_step, end_year, grid_calc,
                                             grid_cap_gen_limit, grid_connect_limit)
    
    onsseter.max_extension_dist(year, time_step, end_year, start_year, grid_calc, max_grid_intensification_cost, min_intensification_load)

    onsseter.pre_selection(eleclimit, year, time_step, 2, auto_intensification)

    
    onsseter.df[SET_LCOE_GRID + "{}".format(year)], onsseter.df[SET_MIN_GRID_DIST + "{}".format(year)], \
        grid_investment, grid_capacity, x_coords, y_coords, new_lines_geojson[year] = \
        onsseter.elec_extension_numba(grid_calc, mv_line_max_length, year, start_year, end_year, time_step, grid_cap_gen_limit, grid_connect_limit,
                                      x_coords, y_coords, mg_interconnection=mg_interconnection, auto_intensification=auto_intensification, 
                                      prioritization=2, threshold=max_grid_intensification_cost,
                                      )
    
    print('Calculating results columns', time.ctime())
    onsseter.results_columns(techs, tech_codes, year, time_step, 2, auto_intensification,
                             mg_interconnection)

    onsseter.calculate_investments_and_capacity(sa_diesel_investment, sa_diesel_capacity, sa_pv_investment,
                                                sa_pv_capacity, mg_diesel_investment, mg_diesel_capacity,
                                                mg_pv_hybrid_investment, mg_pv_hybrid_capacity, mg_wind_investment,
                                                mg_wind_capacity, mg_hydro_investment, mg_hydro_capacity,
                                                grid_investment, grid_capacity, year)

    if year == yearsofanalysis[-1]:
        final_step = True
    else:
        final_step = False

    onsseter.check_grid_limitations(time_step * annual_new_grid_connections_limit, 
                                    time_step * annual_grid_cap_gen_limit * 1000, 
                                    year, time_step, final_step)

    onsseter.apply_limitations(eleclimit, year, time_step, 2, auto_intensification)

    onsseter.calculate_emission(grid_factor=0, year=year,
                                time_step=time_step, start_year=start_year)

    onsseter.df.loc[(onsseter.df[SET_ELEC_FINAL_CODE + '{}'.format(year)] == 1) &
                        (onsseter.df[SET_ELEC_FINAL_CODE + '{}'.format(start_year)] != 1),
                        SET_ELEC_FINAL_CODE + '{}'.format(year)] = 2
    print('')
    
finalize_results(onsseter, yearsofanalysis)

# 6. Results, Summaries and Visualization
With all the calculations and grid-extensions complete, this block gets the final results on which technology was chosen for each point, how much capacity needs to be installed and what it will cost. Then the summaries, plots and maps are generated.

In [ ]:
summary_table, columns = calc_summary_table(onsseter.df)
display(Markdown('### Summary \n These are the summarized results of the selected country'))
summary_table

In [ ]:
bar_plot(summary_table, columns)

In [ ]:
map_plot(onsseter.df)

In [ ]:
print('Number of potential mini-grid locations: ', len(onsseter.df[onsseter.df['FinalElecCode' + "{}".format(end_year)] == 5]))

## 7. Exporting results

This code generates three csv files:
 - one containing all the results for the scenario created
 - one containing the summary for the scenario created
 - one containing some if the key input variables of the scenario

Before we proceed, please write the scenario_name in the first cell below. then move on to the next cell and run it to browse to the directory where you want to save your results. Sample file shall be located at .\ gep-onsset\sample_output. 

**Note that if you do not change the scenario name, the previous output files will be overwritten**

In [ ]:
scenario_name = "default_calib"

In [ ]:
list1 = [('Start_year',start_year,'','',''), 
         ('End_year',end_year,'','',''),
         ('End year electrification rate target',end_electrification_rate_target,'','',''),
         ('Intermediate target year', intermediate_year,'','',''),
         ('Intermediate electrification rate target', intermediate_electrification_rate_target,'','',''),
         ('Urban target tier', urban_target_tier, '', '', ''),
         ('Rural target tier', rural_target_tier, '', '', ''),
         ('Intensification distance', auto_intensification, '', '', 'Distance from existing MV network (km) for automatic connection to the grid'),
         ('Max intensification cost', max_grid_intensification_cost, 'Maximum cost per household (USD/household) for forced grid intensification'),
         ('pop_start_year',pop_start_year,'','','the population in the start year start year'),
         ('pop_end_year',end_year_pop,'','','the projected population in the end year (e.g. 2030)'),
         ('urban_ratio_start_year',urban_ratio_start_year,'','','the urban population population ratio in the start year'),
         ('urban_ratio_end_year',urban_ratio_end_year,'','','the urban population population ratio in the start year'),
         ('elec_ratio_start_year',grid_elec_ratio_start_year,'','','The grid electrification rate in the start year'),
         ('urban_elec_ratio',grid_urban_elec_ratio,'','','The urban electrification rate in the start year'),
         ('rural_elec_ratio',grid_rural_elec_ratio,'','','The rural electrification rate in the start year'),
         ('grid_generation_cost',grid_generation_cost,'','','This is the grid cost electricity USD/kWh as expected in the end year of the analysis'),
         ('grid_power_plants_capital_cost',grid_power_plants_capital_cost,'','','The cost in USD/kW to for capacity upgrades of the grid-connected power plants'),
         ('grid_losses',grid_losses,'','','The fraction of electricity lost in transmission and distribution (percentage)'),
         ('base_to_peak',grid_calc.base_to_peak_load_ratio,'','','The ratio of base grid demand to peak demand (percentage)'),
         ('diesel_price',diesel_price,'','','This is the diesel price in USD/liter as expected in the end year of the analysis'),
         ('mg_hydro_capital_cost',mg_hydro_capital_cost,'','','Mini-grid Hydro capital cost (USD/kW) as expected in the years of the analysis'),
         ('min_mg_size', min_mg_size, 'Minimum number of connections (people) for mini-grids to be considered'), 
         ('mg_min_grid_dist', mg_min_grid_dist, 'Minimum distance from existing MV lines for mini-grids to be considered as an option'), 
         ('mg_interconnection', mg_interconnection, 'Whether mini-grids should be allowed to be interconnected to the grid in a later time-step. 0 = NO, 1 = YES'), 
         ('pv_cost', pv_cost, 'Mini-grid: PV panel costs including BoS (PV inverter, charge controller) (USD/kW)'), 
         ('battery_cost', battery_cost, 'Mini-grid: battery capital cost, USD/kWh of storage capacity'), 
         ('inverter_cost', inverter_cost, 'Mini-grid: Battery inverter (USD/kW)'), 
         ('diesel_gen_cost', diesel_gen_cost, 'Mini-grid: diesel generator capital cost, USD/kW rated power'), 
         ('max_diesel', max_diesel, 'Mini-grid: Maximum share of generation that can come from diesel generators (0-1). Set to 0 for fully renewable mini-grids'), 
         ('sa_pv_capital_cost_1',sa_pv_capital_cost_1,'','','Stand-alone PV capital cost (USD/kW) for household systems under 20 W'),
         ('sa_pv_capital_cost_2',sa_pv_capital_cost_2,'','','Stand-alone PV capital cost (USD/kW) for household systems between 21-50 W'),
         ('sa_pv_capital_cost_3',sa_pv_capital_cost_3,'','','Stand-alone PV capital cost (USD/kW) for household systems between 51-100 W'),
         ('sa_pv_capital_cost_4',sa_pv_capital_cost_4,'','','Stand-alone PV capital cost (USD/kW) for household systems between 101-200 W'),
         ('sa_pv_capital_cost_5',sa_pv_capital_cost_5,'','','Stand-alone PV capital cost (USD/kW) for household systems over 200 W'),
         ('mv_line_cost',mv_line_cost,'','','Cost of MV lines in USD/km'),
         ('lv_line_cost',lv_line_cost,'','','Cost of LV lines in USD/km'),
         ('mv_line_capacity',mv_line_capacity,'','','Capacity of MV lines in kW/line'),
         ('lv_line_capacity',lv_line_capacity,'','','Capacity of LV lines in kW/line'),
         ('lv_line_max_length',lv_line_max_length,'','','Maximum length of LV lines (km)'),
         ('hv_line_cost',hv_line_cost,'','','Cost of HV lines in USD/km'),
         ('max_grid_extension_dist',mv_line_max_length,'','','Maximum distance that the grid may be extended by means of MV lines'),
         ('annual_new_grid_connections_limit', annual_new_grid_connections_limit,'','','This is the maximum amount of new households that can be connected to the grid in one year (thousands)'),
         ('grid_capacity_limit',annual_grid_cap_gen_limit,'','','This is the maximum generation capacity that can be added to the grid in one year (MW)'),
         ('grid_discount_rate', grid_discount_rate, 'Grid discount rate'),
         ('mini_grid_discount_rate',mini_grid_discount_rate , 'Mini-grid discount rate'),
         ('standalone_discount_rate', standalone_discount_rate, 'Stand-alone PV discount rate'),

         ]
labels = ['Variable','Value', 'Source', 'Comments', 'Description']
df_variables = pd.DataFrame.from_records(list1, columns=labels)

In [ ]:
messagebox.showinfo('OnSSET', 'Browse to the folder where you want to save the outputs')

output_dir = filedialog.askdirectory()
output_dir_variables = os.path.join(output_dir, '{}_Variables.csv'.format(scenario_name))
output_dir_results = os.path.join(output_dir, '{}_Results.csv'.format(scenario_name))
output_dir_summaries = os.path.join(output_dir, '{}_Summaries.csv'.format(scenario_name))

In [ ]:
# Returning the result as a csv file
onsseter.df.to_csv(output_dir_results, index=False)

# Returning the summary as a csv file
summary_table.to_csv(output_dir_summaries, index=True)

# Returning the input variables as a csv file
df_variables.to_csv(output_dir_variables, index=False)

In [ ]:
for year in yearsofanalysis:
    try:
        with open(os.path.join(output_dir, 'new_mv_lines_{}_{}.geojson'.format(scenario_name, year)), 'w') as f:
            geojson.dump(new_lines_geojson[year], f)
        gdf = gpd.read_file(os.path.join(output_dir, 'new_mv_lines_{}_{}.geojson'.format(scenario_name, year)))
        gdf = gdf.set_crs(3395, allow_override=True)
        gdf = gdf.to_crs(4326)
        gdf.to_file(os.path.join(output_dir, 'new_mv_lines_{}_{}.geojson'.format(scenario_name, year)))
    except:
        pass